<a href="https://colab.research.google.com/github/worldbank/dec-python-course/blob/main/1-foundations/5-geospatial-analysis/foundations-s5-geo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Geospatial Analysis in Python

## Concepts

### Spatial Data

There are two main types of spatial data: __vector__ and __raster__ data.

Vector Data                    | Raster Data
------------------------------ | -----------
Are points, lines, or polygons | Is a spatially referenced grid
Common file formats include shapefiles (.shp), geojsons (.geojson), etc) | Common file format is a geotif (.tif)
Examples: __Polygons__ of countries, __polylines__ of roads, __points__ of households | Examples: satellite imagery of nighttime lights


Vector           |  Raster
:-------------------------:|:-------------------------:
![polygons](https://github.com/worldbank/dec-python-course/blob/main/1-foundations/5-geospatial-analysis/img/polygons.png?raw=1)  |  ![rasters](https://github.com/worldbank/dec-python-course/blob/main/1-foundations/5-geospatial-analysis/img/raster.png?raw=1)

### Coordinate Reference Systems (CRS)

Coordinate reference systems use pairs of numbers to define a location on the earth. For example, the World Bank is at a latitude of 38.89 and a longitude of -77.04

![wb-loc](https://github.com/worldbank/dec-python-course/blob/main/1-foundations/5-geospatial-analysis/img/google_wb_latlon.png?raw=1)

There are many different coordinate reference systems, which can be grouped into geographic and projected coordinate systems. Geographic systems live on a sphere, while projected systems are "projected" onto a flat surface.

![crs_geo_proj](https://github.com/worldbank/dec-python-course/blob/main/1-foundations/5-geospatial-analysis/img/crs_geo_proj.png?raw=1)

#### Geographic Coordinate Systems

Geographic coordiante systems are represented on a three-dimensional system.

* __Units:__ A point is made up of latitude and longitude, which measure angles (eg, latitude is angle from the equator), and units are typically in decimal degrees.
* __Latitude and Longitude:__ On an X-Y grid, latitude is Y (up/down), and longitude is X (left/right). Consequently, values are often represented as: (longitude, latitude) -- to correspond to (X,Y). However, it has also come convention to represent as (latitude, longitude) -- as latitude comes before longitude in the alphabet. For example, when searching coordinates in Google Maps, values are displayed as (latitude,longitude).
    - Valid values of latitude range from -90 to 90, and valid values of longitude range from -180 to 180.
    - Latitude sounds (and looks like!) like "latter"!
* __Distance on a sphere:__ At the equator (latitude = 0), a 1 decimal degree longitude distance is about 111km; towards the poles (latitude = -90 or 90), a 1 decimal degree longitude distance converges to 0 km. We must be careful (ie, use algorithms that account for a spherical earth) to calculate distances! The distance along a sphere is refered to as a [great circle distance](https://en.wikipedia.org/wiki/Great-circle_distance).
* __Datums:__ The earth is not a perfect sphere, it is a lumpy [20ellipsoid](https://oceanservice.noaa.gov/facts/earth-round.html#:~:text=The%20Earth%20is%20an%20irregularly%20shaped%20ellipsoid.&text=While%20the%20Earth%20appears%20to,unique%20and%20ever%2Dchanging%20shape.) A __datum__ is a model of the earth that is used in mapping (see reference [here](https://www.maptoaster.com/maptoaster-topo-nz/articles/projection/datum-projection.html). One of the most common datums is [WGS 84](https://en.wikipedia.org/wiki/World_Geodetic_System), which is used by the Global Positioning System (GPS). Other datums exist, and the latitude and longitude values for a specific location will be different depending on the datum.

![latlon](https://github.com/worldbank/dec-python-course/blob/main/1-foundations/5-geospatial-analysis/img/latlon.png?raw=1)

#### Projected Coordinate Systems

Projected coordinate systems use a "projection" to flatten data to a 2D surface.
* __Distoritions:__ Projections will distort some combination of distance, area, shape, or direction. Different projects can minimize distorting some aspect at the expense of others. For more on this, see [this clip](https://www.youtube.com/watch?v=eLqC3FNNOaI) from the west wing!
* __Units:__ When projected, points are represented as "northings" and "eastings". Values are often represented in meters, where northings/eastings are the meter distance from some reference point. Consequently, values can be very large (the earth is !

![crs_proj_types](https://github.com/worldbank/dec-python-course/blob/main/1-foundations/5-geospatial-analysis/img/crs_proj_types.png?raw=1)

#### Coordinate Reference System Summary

* __CRSs are important!:__ Whenever we have spatial data, we need to know which coordinate reference system the data is in. You wouldn't say: "I am 5 away"; you would say, "I am 5 [miles / kilometers / minutes] away." Similarly, a "complete" way to describe location would be: "I am at 6.51 latitude, 3.52 longitude using the WGS 1984 CRS. Most issues I see in coding related to GIS stem from coordinate reference system! (The wrong one assinged, related two objects with different CRS, etc).
* __Referencing CRSs:__ Coordinate reference systems can be referenced in multple ways. One standard way of referencing CRSs is defined by [PROJ](https://en.wikipedia.org/wiki/PROJ) (a coordinate reference system library). For example:
    - WGS 84 is referenced as: `+proj=longlat +datum=WGS84 +no_defs +type=crs`
    - Mercator projection is referenced as: `+proj=merc +lat_ts=0 +lon_0=0 +x_0=0 +y_0=0 +R=6371000 +units=m +no_defs +type=crs`
* __Referencing CRSs Using EPSG:__ Referencing CRSs as above can be cumbersome, as the references are long. The European Petroleum Survey Group (EPSG) publishes a [database](https://epsg.io/) of coordinate reference systems that has become common to use for referencing differnet coordinate reference systems. EPSG assigns a numeric ID to each CRS, which can make referencing CRS systems easier.
    - [WGS 84](https://epsg.io/4326) has the following EPSG ID: `4326`
    - [Mercator](https://epsg.io/53004) projection has the following EPSG ID: `53004`
* __For more Info:__ For a more technical overview of coordinate reference systems, see [here](https://r-spatial.org/book/02-Spaces.html).

## Loading and exploring spatial data

### Setup

The main geospatial packages that we'll load are __Shapely__ and __GeoPandas__.

* "__[Shapely](https://pypi.org/project/shapely/)__ is a BSD-licensed Python package for manipulation and analysis of planar geometric objects"
* "__[GeoPandas](https://geopandas.org/en/stable/)__ extends the datatypes used by pandas to allow spatial operations on geometric types. Geometric operations are performed by shapely."

In short, __GeoPandas__ allows processing tabular data (like Pandas), where each row is associated with a geometry---and where the geometry is defined using __Shapely__.

Shapely defines multiple geometry types:

![](https://github.com/worldbank/dec-python-course/blob/main/1-foundations/5-geospatial-analysis/img/shapely_geom_types.png?raw=1)

__Source:__ [Shapely](https://isbe.bwk.tue.nl/education/Python/04_02_Shapely.html#fundamental-geometric-objects)

In [ ]:
#### Required
#! pip install geopandas
#! pip install folium
#! pip install shapely
#! pip install numpy

#### Optional
# ! pip install gadm
# ! pip install osmnx

In [ ]:
import pandas as pd
import os
import geopandas as gpd
from shapely.geometry import Point
import numpy as np

### Load and explore polygon

In [ ]:
# Download administrative region polygons from GADM

if False:
    from gadm import GADMDownloader
    downloader = GADMDownloader(version="4.0")

    nga_0 = downloader.get_shape_data_by_country_name(country_name="Nigeria", ad_level=0)
    nga_0.to_file(os.path.join("data", "gadm_nga_0.geojson"), driver="GeoJSON")

    nga_1 = downloader.get_shape_data_by_country_name(country_name="Nigeria", ad_level=1)
    nga_1.to_file(os.path.join("data", "gadm_nga_1.geojson"), driver="GeoJSON")

    nga_2 = downloader.get_shape_data_by_country_name(country_name="Nigeria", ad_level=2)
    nga_2.to_file(os.path.join("data", "gadm_nga_2.geojson"), driver="GeoJSON")

In [ ]:
nga2_gdf = gpd.read_file(os.path.join("data", "gadm_nga_2.geojson"))

In [ ]:
nga2_gdf.head()

In [ ]:
nga2_gdf.shape

In [ ]:
nga2_gdf.plot()

In [ ]:
nga2_gdf.crs

In [ ]:
nga2_gdf.explore()

#### Area of each polygon

In [ ]:
# Are is not a variable :(
"area" in nga2_gdf.columns.tolist()

In [ ]:
# But we can still figure out the area :)
nga2_gdf.area

In [ ]:
# Lets add the area as a variable
nga2_gdf['area'] = nga2_gdf.area

#### Operations common to pandas dataframes also work on geodataframes

In [ ]:
lagos_gdf = nga2_gdf[nga2_gdf['NAME_1'] == 'Lagos']

In [ ]:
lagos_gdf.plot()

### Exercise 1: Load and Explore Polyline

In [ ]:
## Downloading road network data for Lagos from OpenStreetMaps

if False:

    import osmnx as ox

    #### One geometry for Lagos
    nga1_gdf = gpd.read_file(os.path.join("data", "gadm_nga_1.geojson"))
    lagos_1_gdf = nga1_gdf[nga1_gdf['NAME_1'] == 'Lagos']

    #### Grab larger roads from OSM; ignoring unclassified and residential
    roads_gdf = ox.features.features_from_polygon(lagos_1_gdf.geometry.iloc[0],
                                                        tags = {'highway':['motorway',
                                                                           'trunk',
                                                                           'primary',
                                                                           'secondary',
                                                                           'tertiary']})

    #### Cleanup
    roads_gdf = roads_gdf.reset_index()
    roads_gdf = roads_gdf.drop('nodes', axis=1)

    #### Export
    roads_gdf.to_file(os.path.join("data", "osm_lagos_roads.geojson"), driver="GeoJSON")

#### 1a: Load data

Load the roads data `data/osm_lagos_roads.geojson`, and name the object `roads_gdf`

#### 1b. Look at the first few observations of the data

#### 1c: What is the coordinate reference system of the data?

#### 1d: Make a static map of the road network

#### 1e: Make an interactive map of the road network that just includes motorways and trunk roads

#### 1f: Add a variable indicating the length of each road

### Load and Explore Point Data

In [ ]:
## Downloading school data for Lagos from OpenStreetMaps

if False:

    import osmnx as ox

    #### One geometry for Lagos
    nga1_gdf = gpd.read_file(os.path.join("data", "gadm_nga_1.geojson"))
    lagos_1_gdf = nga1_gdf[nga1_gdf['NAME_1'] == 'Lagos']

    #### Grab larger roads from OSM; ignoring unclassified and residential
    schools_gdf = ox.features.features_from_polygon(lagos_1_gdf.geometry.iloc[0],
                                                        tags = {'amenity':['school']})

    #### Cleanup
    schools_gdf = schools_gdf.reset_index()
    schools_gdf = schools_gdf.drop('nodes', axis=1)

    #### Original is polygon; convert to points using polygon centroid
    schools_gdf['geometry'] = schools_gdf['geometry'].centroid

    #### Add latitude and longitude variables based on centroid
    # (Remove existing Lat/Lon variables, some of which are NA)
    schools_gdf = schools_gdf.drop(['Latitude', 'Longitude'], axis = 1)

    schools_gdf['latitude'] = schools_gdf['geometry'].y
    schools_gdf['longitude'] = schools_gdf['geometry'].x

    #### Drop geometry; convert from geodataframe to dataframe
    schools_df = schools_gdf.drop('geometry', axis=1)

    #### Export
    schools_df.to_csv(os.path.join("data", "osm_lagos_schools.csv"), index = False)

#### Load school data

In [ ]:
schools_df = pd.read_csv(os.path.join("data", "osm_lagos_schools.csv"))

In [ ]:
schools_df.head()

#### Convert from dataframe to geodataframe

In [ ]:
# Combine 'latitude' and 'longitude' columns to create a GeoSeries of Point geometries
geometry = [Point(lon, lat) for lon, lat in zip(schools_df['longitude'], schools_df['latitude'])]
geo_series = gpd.GeoSeries(geometry)

# Convert the Pandas DataFrame to a GeoPandas GeoDataFrame
schools_gdf = gpd.GeoDataFrame(schools_df, geometry=geo_series, crs = 4326)

#### Explore data

In [ ]:
schools_gdf.head()

In [ ]:
schools_gdf.crs

In [ ]:
schools_gdf.explore()

#### Exercise 2: Assign the wrong CRS and see what happens

Above we used the following code to convert the dataframe to a GeoDataFrame, by indicating (1) the data, (2) the geometry, and (3) the CRS:

`schools_gdf = gpd.GeoDataFrame(schools_df, geometry=geo_series, crs = 4326)`

Now, we'll explore what happens if we assign the wrong CRS.

#### 2a. Create an object with the wrong CRS

Adapt the code just above to create a new object called `schools_bad_gdf`; use the CRS `3857`

#### 2b. Make a static map of `schools_bad_gdf`. From this, can we tell that there may be an issue with the CRS?

#### 2b. Make an interactive map of `schools_bad_gdf`. From this, can we tell that there may be an issue with the CRS?

### Maps

In [ ]:
# This is borning.
lagos_gdf.plot()

In [ ]:
# Pretty :)
lagos_gdf.plot(column='area',
               cmap='Spectral',
               legend=True,
               figsize = (20,5))

In [ ]:
lagos_gdf.explore(
     column="area", # make choropleth based on "pings" column
     tooltip="area", # show pings value when hover over
     cmap="Spectral", # use "Spectral" matplotlib colormap
     style_kwds=dict(color="black") # use black outline
    )

#### Exercise 3: Make a different version of the interactive map

1. Use a different color palette (for options, see [here](https://matplotlib.org/stable/users/explain/colors/colormaps.html).
2. When hovering over ADM2 regions, show the name of the region (not the area)

### Spatial Operations: Applied on Single Dataset

#### Transform CRS

We want to compute the length of roads in Lagos using the roads dataset. The roads dataset is currently in a geographic CRS (WGS84), where the units are in decimal degrees. We'll tranform the CRS to a __projected__ CRS that is suitable for Nigeria ([EPSG:32632](https://epsg.io/32632)), and where the units will be in meters.

In [ ]:
# Load data
roads_gdf = gpd.read_file(os.path.join("data", "osm_lagos_roads.geojson"))

In [ ]:
# Length is in decimal degrees. Unhelpful!
roads_gdf.length.head()

In [ ]:
# Use to_crs to change the CRS
roads_newcrs_gdf = roads_gdf.to_crs(32632)

In [ ]:
# Now the length is in meters!
roads_newcrs_gdf.length.head()

Let's look at the correlation between the length in decimal degrees and the length in meters. The correlation super close to 1, but is not exactly 1---demonstrating that the CRS matters when computing lengths, distances, areas, etc. Remember, as latitude changes, the kilometer distance between 1 degree longitude changes. Lagos is all at a very similar latitude, so we'd expect the correlation between length caculated from a geographic and projected CRS to be similar.

In [ ]:
import numpy
numpy.corrcoef(roads_newcrs_gdf.length.to_list(),
               roads_gdf.length.to_list())[0,1]

#### Exercise 4: Transform the CRS of the Nigeria ADM2 object

#### 4a: Transform the CRS of the Nigeria ADM2 object `nga2_gdf` to EPSG:32632

#### 4b: Determine the area in decimal degrees and meters squared

#### 4c: Compute the correlation between the area in decimal degrees and meters squared

#### Buffer

We have the points of schools. Now we create a 1km buffer around schools

In [ ]:
schools_1km_gdf = schools_gdf.copy()
schools_1km_gdf = schools_1km_gdf.to_crs(32632)

schools_1km_gdf['geometry'] = schools_1km_gdf.geometry.buffer(1000)

In [ ]:
schools_1km_gdf.explore()

#### Dissolve by an Attribute

Below we have the second administrative regions of Nigeria. Using this dataset, let's create a new object at the first administrative region level.

In [ ]:
nga1_simple_gdf = nga2_gdf.dissolve('NAME_1')

In [ ]:
nga2_gdf.plot()

In [ ]:
nga1_simple_gdf.plot()

#### Convex Hull

__Simple definition:__ Get the outer-most coordinates of a shape and connect-the-dots.

__Fomal definition:__
A convex hull of a shape the smallest "convex set" that contains it. (A [convex set](https://en.wikipedia.org/wiki/Convex_set) is where a straight line can be drawn anywhere in the space and the space fully contains the line).

In the below example, we create a conex hull around schools; creating a polygon that includes all schools.

__Incorrect attempt:__ We use `convex_hull`, which applies a convex hull to each row. Because schools are points, the resulting geometries are the same (they're still points!)

In [ ]:
schools_gdf['c_hull'] = schools_gdf.convex_hull

schools_gdf['c_hull'].plot()

__Correct attempt:__ We first dissolve the schools into one geometry, then compute the convex hull.

In [ ]:
schools_gdf['id_temp'] = 1
schools_diss_gdf = schools_gdf.dissolve('id_temp')

schools_diss_gdf['c_hull'] = schools_diss_gdf.convex_hull

schools_diss_gdf['c_hull'].plot()

In [ ]:
# Note that the geometry type is a "multipoint"
schools_diss_gdf.head()

#### Determine Centroid

Sometimes we want to use the centroid of a point or polygon. Below shows an example of creating a geodataframe of Nigeria's ADM2, where we use the centroid as the geometry.

In [ ]:
nga2_c_gdf = nga2_gdf.copy()
nga2_c_gdf['geometry'] = nga2_c_gdf['geometry'].centroid

nga2_c_gdf.plot()

#### Exercise 5: Create a roads object where (1) roads are buffered by 10 meters and (2) road types are dissolved, so the dataset contains one row per road type

#### 5a. Buffer the road network by 10 meters

#### 5b. Dissolve the road network by road type (ie, the "highway" variable)

### Spatial Operations: Using Multiple Datasets

#### Distances

For this example, we'll compute the distance between each school to a motorway.

In [ ]:
#### Project so units are in meters
schools_proj_gdf = schools_gdf.to_crs(32632)
roads_proj_gdf   = roads_gdf.to_crs(32632)

In [ ]:
#### Road dataset of just motorways
roads_diss_gdf = roads_proj_gdf.dissolve('highway')
roads_diss_gdf = roads_diss_gdf.reset_index()

roads_motorway_gdf = roads_diss_gdf[roads_diss_gdf['highway'] == 'motorway']

In [ ]:
#### Distance
schools_proj_gdf['dist_to_road_m'] = schools_proj_gdf['geometry'].apply(lambda x: x.distance(roads_motorway_gdf['geometry'].iloc[0]))

schools_proj_gdf['dist_to_road_m'].describe()

#### Intersects (True/False vector)

For this example we'll determine which of Lagos's second administrative divisions intersects with a motorway.

First, let's check the CRS of the datasets we'll use and ensure that they're in the same CRS.

In [ ]:
roads_motorway_gdf.crs

In [ ]:
lagos_gdf.crs

In [ ]:
lagos_proj_gdf = lagos_gdf.to_crs(32632)

In [ ]:
## Create new variable whether ADM intersects with motorway
lagos_proj_gdf['inter_motorway'] = lagos_proj_gdf['geometry'].apply(lambda x: x.intersects(roads_motorway_gdf['geometry'].iloc[0]))

In [ ]:
lagos_proj_gdf.inter_motorway.head()

#### Overlay Data

We have roads for all of Lagos. Here we'll show how to use the `overlay` function to:

1. Create a new object of roads that are in a certain administrative region.
2. Create a new object of roads that are NOT in a certain administrative region.

In [ ]:
eo_proj_gdf = lagos_proj_gdf[lagos_proj_gdf.NAME_2 == "Eti-Osa"]

In [ ]:
## 1. Create a new object of roads that are in a certain administrative region.
roads_eo_gdf = roads_proj_gdf.overlay(eo_proj_gdf, how='intersection')

roads_eo_gdf.plot()

In [ ]:
## 2. Create a new object of roads that are NOT in a certain administrative region.
roads_not_eo_gdf = roads_proj_gdf.overlay(eo_proj_gdf, how='difference')

roads_not_eo_gdf.plot()

#### Spatial Join

We have a dataset of schools. The school GeoDataFrame contains information such as the school name, but not on the administrative region it's in. To add data on the administrative region that the school is in, we'll perform a spatial join

In [ ]:
schools_admdata_gdf = gpd.sjoin(
    schools_gdf,
    nga2_gdf,
    how='left',
    predicate="within",
)

schools_admdata_gdf.head()

#### Exercise 6: Determine the ADM2 name that the centroid of each road is in

#### 6a: Create a new roads object where each road geometry is the centroid of the road

#### 6b: Determine the ADM2 name that each road centroid is in

## Challenge Exercise

__Inaccessible schools: Create an interactive map of Lagos second administrative divisions that shows the number of schools that are greater than 1 kilometer of a trunk road or motorway__

In [ ]:
## Load data
nga2_gdf  = gpd.read_file(os.path.join("data", "gadm_nga_2.geojson"))
roads_gdf = gpd.read_file(os.path.join("data", "osm_lagos_roads.geojson"))

# Additional Resources

* https://aeturrell.github.io/coding-for-economists/geo-intro.html
* https://pygis.io/docs/d_access_osm.html